# GPU Version of Fractal Generation

In [26]:
amps = {'0': (0.05334708691207961+0.02209708691207961j), '1': (0.1923456572581369+0.07967217989988726j), '10': (0.07544417382415922+0.03125j), '11': (0.15088834764831843+0.0625j), '100': (0.2502200171930884+0.10364452469860624j), '101': (0.1686983011771303+0.06987712429686843j), '110': (0.15088834764831843+0.0625j), '111': (0.2970240093698727+0.12303137303143455j), '1000': (0.23857542547399874+0.09882117688026186j), '1001': (0.2502200171930884+0.10364452469860624j), '1010': (0.17693227094584735+0.07328774624724109j), '1011': (0.10669417382415922+0.04419417382415922j), '1100': (0.07544417382415922+0.03125j), '1101': 0j, '1110': (0.05334708691207961+0.02209708691207961j), '1111': (0.10669417382415922+0.04419417382415922j), '10000': (0.07544417382415922+0.03125j), '10001': (0.11928771273699937+0.04941058844013093j), '10010': 0j, '10011': (0.07544417382415922+0.03125j), '10100': (0.10669417382415922+0.04419417382415922j), '10101': (0.11928771273699937+0.04941058844013093j), '10110': (0.1600412607362388+0.06629126073623882j), '10111': 0j, '11000': (0.18479972993502913+0.07654655446197431j), '11001': (0.1923456572581369+0.07967217989988726j), '11010': (0.10669417382415922+0.04419417382415922j), '11011': 0j, '11100': (0.09239986496751457+0.038273277230987154j), '11101': 0j, '11110': (0.09239986496751457+0.038273277230987154j), '11111': (0.09239986496751457+0.038273277230987154j), '100000': (0.09239986496751457+0.038273277230987154j), '100001': (0.15088834764831843+0.0625j), '100010': (0.10669417382415922+0.04419417382415922j), '100011': 0j, '100100': (0.2444670638799928+0.10126157341262282j), '100101': (0.05334708691207961+0.02209708691207961j), '100110': (0.09239986496751457+0.038273277230987154j), '100111': (0.13067314219850173+0.05412658773652741j), '101000': (0.05334708691207961+0.02209708691207961j), '101001': (0.1411431251391113+0.05846339666834283j), '101010': (0.10669417382415922+0.04419417382415922j), '101011': (0.05334708691207961+0.02209708691207961j), '101100': (0.07544417382415922+0.03125j), '101101': 0j, '101110': (0.13067314219850173+0.05412658773652741j), '101111': (0.13067314219850173+0.05412658773652741j), '110000': (0.10669417382415922+0.04419417382415922j), '110001': (0.10669417382415922+0.04419417382415922j), '110010': (0.11928771273699937+0.04941058844013093j), '110011': (0.09239986496751457+0.038273277230987154j), '110100': (0.1411431251391113+0.05846339666834283j), '110101': (0.10669417382415922+0.04419417382415922j), '110110': (0.10669417382415922+0.04419417382415922j), '110111': (0.1923456572581369+0.07967217989988726j), '111000': (0.1686983011771303+0.06987712429686843j), '111001': 0j, '111010': (0.05334708691207961+0.02209708691207961j), '111011': (0.10669417382415922+0.04419417382415922j), '111100': (0.3064556827767503+0.1269381000724369j), '111101': (0.27719959490254376+0.11481983169296148j), '111110': 0j, '111111': (0.3156056223678123+0.1307281291459493j)}

In [2]:
! pip install celluloid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from celluloid import Camera
import torch
import tqdm

c = 1.4 + 0.2j

print("c= ",c)

# Define the size
size = 1000
heightsize = size
widthsize = size

def julia_set(c=c, height=heightsize, width=widthsize, x=0, y=0, zoom=1, max_iterations=100, amps=amps):
    # To make navigation easier we calculate these values
    x_width = 1.5
    y_height = 1.5*height/width
    x_from = x - x_width/zoom
    x_to = x + x_width/zoom
    y_from = y - y_height/zoom
    y_to = y + y_height/zoom
    
    # Here the actual algorithm starts and the z paramter is defined for the Julia set function
    x = torch.tensor(np.linspace(x_from, x_to, width).reshape((1, width)), device='cuda')
    y = torch.tensor(np.linspace(y_from, y_to, height).reshape((height, 1)), device='cuda')
    z = x + 1j * y
    
    # Initialize c to the complex number obtained from the quantum circuit
    c = torch.tensor(np.full(z.shape, c), device='cuda')

    # To keep track in which iteration the point diverged
    div_time = torch.tensor(np.zeros(z.shape, dtype=int), device='cuda')
    
    # To keep track on which points did not converge so far
    m = torch.tensor(np.full(c.shape, True, dtype=bool), device='cuda')
    
    for key in amps:
        amps[key] = torch.tensor(np.full(z.shape, amps[key]), device='cuda')
    
    for i in range(max_iterations):
        num = z[m]**4 + sum([amps[bin(num)[2:]][m] * z[m] ** ((num // 2)  % 4) for num in range(len(amps.keys()), 2)])
        den = z[m]**4 + sum([amps[bin(num)[2:]][m] * z[m] ** (((num - 1) // 2) % 4) for num in range(1, len(amps.keys()), 2)])
        # print(num.shape)
        # print(den.shape)
        z[m] = num / den + c[m]
        m[torch.abs(z) > 2] = False
        div_time[m] = i
    # print(div_time.max())
    # print(div_time.min())
    return div_time.cpu().numpy()


fig, ax = plt.subplots(1, 1) 
camera = Camera(fig)
def color(data, i):
    normalizedData = (data - np.min(data)) / (np.max(data) - np.min(data))
    normi = i/(frameno)
    # print(data.shape)
    rgb = np.zeros((data.shape[0], data.shape[1], 3))
    r = normalizedData*normi
    b = (1-normi)*normalizedData
    # print(r,b)
    rgb[:,:,0] = r#np.full(rgb[:,:,1].shape, 0)
    rgb[:,:,1] = np.full(rgb[:,:,1].shape, 0)
    rgb[:,:,2] = b#normalizedData
    return rgb


frameno = 50
for i in tqdm.tqdm(range(frameno)):
    map = julia_set(c=c, amps=amps.copy())
    data = color(map, i)
    ax.imshow(data)
    ax.axis('off')
    camera.snap()
    plt.close()
    c += 0.03 + 0.03j
    
anim = camera.animate(blit=True)
anim.save('1qubit_simulator_4animations_H_2' + str(frameno) + ' steps c=' + str(c) + '.gif', writer='pillow')

c=  (1.4+0.2j)


100%|██████████| 50/50 [00:54<00:00,  1.09s/it]
